Diego Toribio <br>
Professor Fred Fontaine <br>
EID-378 Finance <br>
Problem Set II: Interest Rates and Bonds <br>

In [1]:
import numpy as np

## Section 1

The following code will perform backsubstitution: if $ A $ is a lower triangular matrix with nonzero elements on the diagonal, this will solve $ y = Ax $. Here, $ n $ is the length of the $ x,y $ vectors $( A $ is $ n \times n $):

```python
import numpy as np

x = np.zeros(n)

for i in range(0, n):
    tmp = y[i]
    for j in range(0, i):
        tmp -= x[j] * A[i, j]

    x[i] = tmp / A[i, i]
```

Feel free to modify it, convert it to a function, etc. It does not do error checking, and is based on the premise that $ A[i, j] = 0 $ for $ j > i $ (the code will still run if not, it just won’t solve $ y = Ax $ correctly), and that $ A[i, i] \neq 0 $.

Write Python code to do the following. Assume spot rates $ r $ and forward rates $ f $ are in continuous-compounding form, for simplicity. Given $ N $ bonds with prescribed coupon rates (coupons payable semi-annually), face value, current prices, and maturities exactly $ 0.5m $ years from today (i.e., coinciding with the coupon payments), with $ m = 1, 2, \dots, N $. You can have these values stored in whatever form you prefer—in separate `numpy.array` (vectors) or all together in a dataframe; note the above code, however, assumes $ y $ and $ A $ are `numpy` arrays.

(a) Solve for the discount factors $ Z(0, 0.5m) $ and from that term structure, i.e., $ r(0, 0.5m) $. Specifically, set up the matrix and use backsubstitution to solve for the term structure.

(b) From the term structure, compute the forward rates $ f(0, 0.5m, 0.5m + 0.5) $ for $ 0 \leq m \leq N - 1 $.

(c) Now apply your code to the following problem, assuming face value $\$1000$ for each bond:

| Bond Name | A      | B      | C      | D      |
|-----------|--------|--------|--------|--------|
| Maturity  | 0.5    | 1      | 1.5    | 2      |
| Bond Price| 985.86 | 974.70 | 967.99 | 966.82 |
| Coupon    | 2%     | 3%     | 4%     | 5%     |

(d) Plot the term structure (as a continuous curve), and display the forward rates.


### 1.1 - Discount Factor Calibration

In this part, we want to find the discount factors $Z(0, 0.5m)$ for $m = 1, 2, \dots, N$ by solving a lower-triangular system of linear equations that come from the pricing of $N$ bonds. Each bond has a known market price and semiannual coupon payments (plus face value at maturity).

- **Setup**  
  -  Each bond’s price equals the present value of its cash flows.  
  - The cash flows are semiannual coupons (based on the annual coupon rate, split in two payments per year) and the face value redemption at maturity.  
  - If we denote the discount factor at time $0.5m$ by $Z(0,0.5m)$, then each bond’s price imposes one equation.  
  - Arranging these equations for $N$ bonds in matrix form gives us:
     $$
        A \times x = y,
     $$
     where 
     - $x$ is the vector of unknown discount factors $\bigl[ Z(0,0.5),\,Z(0,1),\,\dots,Z(0,0.5N)\bigr]$,  
     - $A$ is a lower-triangular matrix (since the first bond’s price equation involves only $Z(0,0.5)$, the second bond’s equation involves $Z(0,0.5)$ and $Z(0,1)$, etc.),  
     - $y$ is the vector of bond prices adjusted by their coupon amounts.

- **Solving via Back-Substitution**  
  Once $A$ and $y$ are set up, we can solve for $x$ via the standard forward- or back-substitution approach for triangular systems. Below, we write a small helper function `solve_lower_triangular` that uses the code snippet given in the problem statement to solve $A\,x = y$.

- **Spot Rates**  
  After we find each $Z(0,0.5m)$, we convert it to a continuously-compounded spot rate:
  $$
    r\bigl(0, 0.5m\bigr) \;=\; -\frac{1}{0.5m}\,\ln\!\Bigl(Z\bigl(0, 0.5m\bigr)\Bigr).
  $$

In [ ]:
import numpy as np

def solve_lower_triangular(A, y):
    """
    Solves A x = y for x using back-substitution,
    assuming A is lower-triangular with nonzero diagonal entries.
    Returns x as a 1D NumPy array.
    """
    n = A.shape[0]
    x = np.zeros(n)
    for i in range(n):
        tmp = y[i]
        for j in range(i):
            tmp -= A[i, j] * x[j]
        x[i] = tmp / A[i, i]
    return x

def build_system(prices, coupons, face_value):
    """
    Constructs the system A x = y for discount factors.
    
    Parameters:
        prices: List or array of bond prices. Its length defines N.
        coupons: List of lists, where each inner list contains the coupon payments 
                 for that bond at times 0.5, 1.0, ..., 0.5*(i+1) for the i-th bond.
        face_value: The redemption amount paid at maturity.
        
    Returns:
        A: The lower-triangular coefficient matrix.
        y: The vector of bond prices.
    """
    N = len(prices)  # Number of bonds is determined by the length of the prices list
    A = np.zeros((N, N))
    y = np.array(prices)
    
    for i in range(N):
        # For bond i, there are (i+1) cash flows at times 0.5, 1.0, ..., 0.5*(i+1)
        for j in range(i + 1):
            # For the final cash flow, add the face value to the coupon payment
            if j == i:
                A[i, j] = coupons[i][j] + face_value
            else:
                A[i, j] = coupons[i][j]
    return A, y

# Define bond data (you can change these lists to have any number of bonds)
prices = [95.0, 92.0, 90.0]  # Market prices for bonds 1, 2, and 3 (N = 3 here)
coupons = [
    [2.0],          # Bond 1: 1 cash flow at 0.5 years
    [2.5, 2.5],     # Bond 2: cash flows at 0.5 and 1.0 years
    [3.0, 3.0, 3.0] # Bond 3: cash flows at 0.5, 1.0, and 1.5 years
]
face_value = 100.0

# Build the system for N bonds
A, y_vec = build_system(prices, coupons, face_value)

# Solve for discount factors: x = [Z(0,0.5), Z(0,1.0), ..., Z(0,0.5*N)]
Z = solve_lower_triangular(A, y_vec)

# Maturities for each discount factor: 0.5, 1.0, ..., 0.5*N
N = len(prices)  # N is dynamically determined here
maturities = np.array([0.5 * (i+1) for i in range(N)])

# Convert discount factors to continuously compounded spot rates:
# r(0, T) = - (1/T) * ln(Z(0,T))
spot_rates = -np.log(Z) / maturities

# Output the results
print("Matrix A:")
print(A)
print("\nVector y (Prices):")
print(y_vec)
print("\nDiscount Factors:", Z)
print("Spot Rates (continuous compounding):", spot_rates)

Matrix A:
[[102.    0.    0. ]
 [  2.5 102.5   0. ]
 [  3.    3.  103. ]]

Vector y (Prices):
[95. 92. 90.]

Discount Factors: [0.93137255 0.87484457 0.82117814]
Spot Rates (continuous compounding): [0.14219184 0.13370904 0.13134347]


## Section 2

Refer to the Python code below. It will compute the YTM from information about a bond (face value, price, and coupon rate).

```python
## From Mastering Python in Finance (with typo corrections by FF)
import scipy.optimize as optimize

def bond_ytm(price, FaceVal, T, coup, freq=2, guess=0.05):
    freq = float(freq)
    periods = T * freq
    coupon = coup / 100. * FaceVal / freq
    dt = [(i + 1) / freq for i in range(int(periods))]
    ytm_func = lambda y: sum([coupon / (1 + y / freq)**(freq * t) 
                              for t in dt]) + FaceVal / (1 + y / freq)**periods - price
    return optimize.newton(ytm_func, guess)
```

Write Python code that: given the term structure (annual) $ r_1(0,m) $, $ 1 \leq m \leq N $, and **annual** coupon rate (i.e., assume the coupons are paid annually) of a bond, will compute the price per $\$1$ face value and YTM of the bond.

Now take $ N = 10 $ and assume the Nelson-Siegel model with parameters $ \beta_0 = 0.02 $, $ \beta_1 = 0.02 $, $ \beta_2 = 0.20 $, $ \tau = 5 $:

$$
r_1(0,T) = \beta_0 + (\beta_1 + \beta_2)\frac{\tau_1}{T}\left(1 - e^{-T/\tau_1}\right) - \beta_2 e^{-T/\tau_1}
$$

Compute the price per $\$1$ face value and YTM for coupon rates $ 0\%, 1\%, \dots, 9\% $, and plot each curve.


## Section 3

The Macauley duration $ D_{mac} $ assuming annual compounding can be expressed as:

$$
D_{mac} = 1 + \frac{1}{y_1} + \frac{T\,(y_1 - c) - (1 + y_1)}{c\left[(1+y_1)^T - 1\right] + y_1}
$$

where $ y_1 $ is the YTM, $ c $ is the coupon rate, and $ T $ is time-to-maturity in years. As a hint to its behavior, regardless of $ c $:

$$
\lim_{T \to \infty} D_{mac} = 1 + \frac{1}{y_1}
$$

(a) Write a function in Python to compute $ D_{mac} $ from these three parameters.

(b) As an example, set $ y_1 = 10\% $ and graph duration as a function of time to maturity, up to $ T = 100 $ years, for $ c = 2\% $ and $ 10\% $ (superimposed) with a horizontal line indicating the limiting value $ D_{mac}(T = \infty) $. [The reason for going out to 100 years is to show the convergence.] This should replicate a graph in the Brandimarte text.

(c) Generate several plots to help us visualize how $ D_{mac} $ varies with these parameters. What you do is up to you. We don’t want 1,000 plots, and the plots should have reasonable values (the above was an exception—don’t take $ T > 30 $ years normally). Do whatever you think is reasonable to illustrate how $ D_{mac} $ varies with these parameters.


## Section 4

Companies $ A $ and $ B $ have been offered the following rates per year on a $\$1$ million, 5-year investment:

|           | Fixed Rate | Floating Rate |
|-----------|------------|---------------|
| **A**     | 8.8%       | LIBOR         |
| **B**     | 8.0%       | LIBOR         |

(a) Company $ A $ prefers a fixed-rate loan, and company $ B $ prefers a floating-rate loan. Bank $ X $ has been engaged as an intermediary for a swap. The swap should be equally attractive to each company, and the bank should earn 0.2% annually. Design the swap.

(b) Suppose instead company $ A $ were offered a fixed rate of 8.0% and company $ B $ a rate of 8.8%. If you repeat your calculation, you will find a problem with it, and neither company would actually engage in the transaction. Explain in *words*, based on the discussion in the lecture: why doesn’t a swap make sense here?